# Sabiá e HuggingFace

In [ ]:
!pip install transformers peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 16.8 MB/s eta 0:00:00


In [ ]:
!python -m pip install bitsandbytes --prefer-binary --extra-index-url=https://jllllll.github.io/bitsandbytes-windows-webui
!pip install accelerate
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://jllllll.github.io/bitsandbytes-windows-webui


In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
from peft import PeftModel, PeftConfig

tokenizer = LlamaTokenizer.from_pretrained("maritaca-ai/sabia-7b")
model = LlamaForCausalLM.from_pretrained(
    "maritaca-ai/sabia-7b",
    device_map="auto",  # Automatically loads the model in the GPU, if there is one. Requires pip install acelerate
    low_cpu_mem_usage=True,
    llm_int8_enable_fp32_cpu_offload=True   # If your GPU does not support bfloat16, change to torch.float16
)

prompt = """Classifique a resenha de filme como "positiva" ou "negativa".

Resenha: Gostei muito do filme, é o melhor do ano!
Classe: positiva

Resenha: O filme deixa muito a desejar.
Classe: negativa

Resenha: Apesar de longo, valeu o ingresso.
Classe:"""

input_ids = tokenizer(prompt, return_tensors="pt")

output = model.generate(
    input_ids["input_ids"].to("cuda"),
    max_length=1024,
    eos_token_id=tokenizer.encode("\n"))  # Stop generation when a "\n" token is dectected

# The output contains the input tokens, so we have to skip them.
output = output[0][len(input_ids["input_ids"][0]):]

print(tokenizer.decode(output, skip_special_tokens=True))

In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [25]:
import os

from openai import OpenAI

In [26]:
# Import Colab Secrets userdata module
from google.colab import userdata

# Set OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

In [27]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [30]:
def chat_with_gpt(prompt, model):
  response = client.chat.completions.create(
    model=model,  # Or another suitable engine
    messages=[
      {"role": "system", "content": "You are ChatGPT."},
      {"role": "user", "content": prompt}
     ],

    max_tokens=150,  # Adjust as needed
    n=1,
    stop=None,
    temperature=0.7,  # Adjust for creativity
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0  # Adjust for diversity
  )
  return response.choices[0].message.content



In [32]:
model_engine = "gpt-4o-mini"
prompt = ""
while True:
  user_input = input("You: ")
  prompt += user_input + "\n"
  if user_input.lower() == 'exit':
    break
  response = chat_with_gpt(prompt, model_engine)
  prompt += "\n" + response + "\n"
  print("ChatGPT: ", response)

You: OI
ChatGPT:  Oi! Como posso ajudar você hoje?
You: meu nome é Aline
ChatGPT:  Oi, Aline! É um prazer te conhecer. Como posso ajudar você hoje?
You: Você lembra meu nome?
ChatGPT:  Sim, Aline! Eu me lembro do seu nome. Como posso ajudar você hoje?
You: acho que você não pode me ajudar
ChatGPT:  Sinto muito se não consigo ajudar da maneira que você precisa. Mas estou aqui para ouvir e tentar ajudar da melhor forma possível. Se quiser, pode compartilhar o que está pensando!
You: exit
